In [ ]:
import selective_scan_cuda

In [1]:
from mamba_ssm.modules.mamba_simple import Mamba

import torch
# from mamba_ssm import Mamba
from torchinfo import summary
# from torchprofile import profile_macs

batch, length, dim = 7, 137, 57
print("B,L,D:",batch, length, dim)
x = torch.randn(batch, length, dim).to("cuda")
model = Mamba(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=64,  # SSM state expansion factor
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to("cuda")
model.eval()

# summary_str = summary(model, input_size=[(7, 201, 286), (7, 201, 286)], depth=5, col_names=("input_size", "output_size", "num_params"), verbose=0)
summary_str = summary(model, input_size=x.shape, depth=15, col_names=("input_size", "output_size", "num_params"), verbose=0)
print(summary_str)

# y = model(x)
# print(x.shape)
# print(y.shape)
# assert y.shape == x.shape

B,L,D: 7 137 57
------ class Mamba in mamba_simple.py -------
1111111111111111111111111111111111111111
------ class MambaInnerFn in selective_scan_interface.py ------
------ selective_scan_cuda.fwd ------
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Mamba                                    [7, 137, 57]              [7, 137, 57]              43,092
Total params: 43,092
Trainable params: 43,092
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0
Input size (MB): 0.22
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.22


In [ ]:
from mamba_ssm.modules.mamba_simple import Mamba

import torch
# from mamba_ssm import Mamba
# from torchinfo import summary
from torchprofile import profile_macs


batch, length, dim = 7, 137, 57
print("B,L,D:",batch, length, dim)
x = torch.randn(batch, length, dim).to("cuda")
model = Mamba(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=64,  # SSM state expansion factor
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to("cuda")
model.eval()

# 使用 torchprofile 計算 FLOPs
macs = profile_macs(model, x)
flops = macs * 2  # MACs (Multiply-Accumulate) 通常乘以 2 轉換為 FLOPs

# 計算參數量
param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)

# 輸出結果
print(f"FLOPs: {flops / 1e9:.3f} GFLOPs")
print(f"參數量: {param_count / 1e6:.3f} M")


# summary_str = summary(model, input_size=[(7, 201, 286), (7, 201, 286)], depth=5, col_names=("input_size", "output_size", "num_params"), verbose=0)
# summary_str = summary(model, input_size=[(batch, length, dim)], depth=15, col_names=("input_size", "output_size", "num_params"), verbose=0)
# print(summary_str)

# y = model(x)
# print(x.shape)
# print(y.shape)
# assert y.shape == x.shape

In [ ]:
model.TSMamba

In [ ]:
import torch

class SquareFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input_tensor):
        # 儲存 input 給 backward 使用
        ctx.save_for_backward(input_tensor)
        # 自訂屬性範例（非 tensor）
        ctx.note = "我是 forward 儲存的小抄"
        print(ctx.note)
        return input_tensor ** 2

    @staticmethod
    def backward(ctx, grad_output):
        # 取出在 forward 存下來的 tensor
        (input_tensor,) = ctx.saved_tensors
        print("小抄內容：", ctx.note)
        # 根據 y = x^2，dy/dx = 2x
        grad_input = 2 * input_tensor * grad_output
        return grad_input

# 用我們自訂的 Function 包裝一下
square = SquareFunction.apply

# 建立一個需要 gradient 的 tensor
x = torch.tensor([3.0], requires_grad=True)

# 呼叫自訂的 forward
y = square(x)

# 執行 backward
y.backward()

# 看 gradient
print("x 的 gradient：", x.grad)